To Do:

I need to create a Syft equivalent of nn.Module that lets you define a model if you initilize it with all the layers it will use, and define its forward pass using the activation functions.

SyMPC seems to make a Module equivalent but for each type of layer- i.e. Conv2d is implemented as a Module, and (I think) their MPCTensor tracks all its gradients.

Perhaps this will come down to modifying the loss function to take the DP Tensor as input, but I think if we use `publish` in order to do the conversion from DP Tensor -> array/tensor, we wouldn't have to do this modification


In [1]:
import syft

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import numpy as np

In [3]:
from syft import nn

In [4]:
import syft.core.tensor.nn.functional as F

In [5]:
from typing import Union
from typing import Sequence
from typing import Tuple, Optional
from syft import PhiTensor
from syft.core.tensor.nn.conv_layers import child_to_torch
from torch import Tensor
from syft.core.adp.data_subject_list import DataSubjectList as DSL

class Conv2d(torch.nn.Module):
    
    def __init__(self, in_channels:int, out_channels:int, kernel_size: Union[int, Sequence[int]], padding:int):
        super(Conv2d, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.func = torch.nn.Conv2d(in_channels=self.in_channels, out_channels=self.out_channels, kernel_size=self.kernel_size, padding=self.padding)
        
    def forward(self, image: PhiTensor):
        # TODO: This conversion is only required the first time and not after that.
        torch_tensor = child_to_torch(image)
        data = self.func(torch_tensor)
        data_array = data.detach().numpy()

        return PhiTensor(
            child=data_array,
            data_subjects=DSL(
                one_hot_lookup=image.data_subjects.one_hot_lookup,
                data_subjects_indexed=np.zeros_like(data_array),
            ),
            min_vals=data_array.min(),
            max_vals=data_array.max(),
        )
#             return nn.Conv2d(
#                 image=x, 
#                 in_channels=self.in_channels, 
#                 out_channels=self.out_channels, 
#                 kernel_size=self.kernel_size, 
#                 padding=self.padding
#             )
    
    def parameters(self):
        return self.func.parameters()
    
    
class BatchNorm2d(torch.nn.Module):
    
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True):
        super(BatchNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.func = torch.nn.BatchNorm2d(num_features=self.num_features, 
            eps=self.eps, 
            momentum=self.momentum, 
            affine=self.affine
        )
        
    def forward(self, image: PhiTensor):
#         return nn.BatchNorm2d(
#             image=x, 
#             num_features=self.num_features, 
#             eps=self.eps, 
#             momentum=self.momentum, 
#             affine=self.affine
#         )
        data = self.func(Tensor(image.child.decode()))
        minv, maxv = image.min_vals.data, image.max_vals.data
        public_avg = 0.5 * (maxv + minv)
        # Assumption: max and min are equally distant from public_avg -> var = 0.25 * (max - min)**2
        public_var = 0.25 * (maxv - minv) ** 2

        return PhiTensor(
            child=data.detach().numpy(),
            data_subjects=image.data_subjects,
            min_vals=(minv - public_avg) / np.sqrt(public_var + self.eps),
            max_vals=(maxv - public_avg) / np.sqrt(public_var + self.eps),
        )
    
    
    def parameters(self):
        return self.func.parameters()

class MaxPool2d(torch.nn.Module):
    
    def __init__(self, kernel_size: Union[int, Tuple[int, int]],
                 stride: Optional[Union[int, Tuple[int, int]]] = None,
                 padding: Union[int, Tuple[int, int]] = 0,
                 dilation: int = 1,
                 return_indices: bool = False,
                 ceil_mode: bool = False,
                ):
        super(MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.return_indices = return_indices
        self.ceil_mode = ceil_mode
        self.func = torch.nn.MaxPool2d(kernel_size=self.kernel_size, 
            stride=self.stride, 
            padding=self.padding, 
            return_indices=self.return_indices, 
            ceil_mode=self.ceil_mode)
        
    def forward(self, image: PhiTensor):
        data = self.func(Tensor(image.child.decode())).detach().numpy()
        return PhiTensor(
            child=data,
            data_subjects=DSL(
                one_hot_lookup=image.data_subjects.one_hot_lookup,
                data_subjects_indexed=np.zeros_like(data),
            ),
            min_vals=np.ones_like(data) * image.min_vals.data,
            max_vals=np.ones_like(data) * image.max_vals.data,
        )
#         return nn.MaxPool2d(
#             image=x, 
#             kernel_size=self.kernel_size, 
#             stride=self.stride, 
#             padding=self.padding, 
#             return_indices=self.return_indices, 
#             ceil_mode=self.ceil_mode
#         )
    
    def parameters(self):
        return self.func.parameters()
    
    
class AvgPool2d(torch.nn.Module):
    def __init__(
        self, 
        kernel_size: Union[int, Tuple[int, int]], 
        stride: Optional[Union[int, Tuple[int, int]]] = None, 
        padding: Union[int, Tuple[int, int]] = 0
    ):
        super(AvgPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.func = torch.nn.AvgPool2d(kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
        
    def forward(self, image: PhiTensor):
        data = self.func(Tensor(image.child.decode())).detach().numpy()
        return PhiTensor(
            child=data,
            data_subjects=DSL(
                one_hot_lookup=image.data_subjects.one_hot_lookup,
                data_subjects_indexed=np.zeros_like(data),
            ),
            min_vals=np.ones_like(data) * image.min_vals.data,
            max_vals=np.ones_like(data) * image.max_vals.data,
        )
        
#         return nn.AvgPool2d(image=x, kernel_size=self.kernel_size, stride=self.stride, padding=self.padding)
    
    def parameters(self):
        return self.func.parameters()

    
class Linear(torch.nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.bias = bias
        self.func = torch.nn.Linear(in_features=self.in_features, out_features=self.out_features, bias=self.bias)
    
    def forward(self, image: PhiTensor):
        image_asarray = image.child.decode()
        data = self.func(Tensor(image_asarray)).detach().numpy()
        minv = (
            self.func(Tensor(np.ones_like(image_asarray) * image.min_vals.data))
            .detach()
            .numpy()
        )
        maxv = (
            self.func(Tensor(np.ones_like(image_asarray) * image.max_vals.data))
            .detach()
            .numpy()
        )

        return PhiTensor(
            child=data,
            data_subjects=DSL(
                one_hot_lookup=image.data_subjects.one_hot_lookup,
                data_subjects_indexed=np.zeros_like(data),
            ),
            min_vals=minv,
            max_vals=maxv,
        )
        
#         return nn.Linear(image=x, in_features=self.in_features, out_features=self.out_features, bias=self.bias)
    
    def parameters(self):
        return self.func.parameters()

In [6]:
class CrossEntropyLoss(torch.nn.Module):
    def __init__(self, weight: Optional[Tensor] = None, size_average=None, ignore_index: int = -100,
                 reduce=None, reduction: str = 'mean'):
        super(CrossEntropyLoss, self).__init__()
        self.weight = weight
        self.size_average = size_average
        self.ignore_index = ignore_index
        self.reduce = reduce
        self.reduction = reduction
        self.func = torch.nn.CrossEntropyLoss(
            self.weight, self.size_average, self.ignore_index,
            self.reduce, self.reduction
        )
        
    def forward(self, input: PhiTensor, target: PhiTensor):
        input_asarray = input.child.decode()
        target_asarray = target.child.decode()
        
        data = self.func(Tensor(input_asarray), Tensor(target_asarray).long()).detach().numpy()
        
        minv = (
            self.func(
                Tensor(np.ones_like(input_asarray) * input.min_vals.data),
                Tensor(np.ones_like(target_asarray) * target.min_vals.data).long(),
            )
            .detach()
            .numpy()
        )
        maxv = (
            self.func(
                Tensor(np.ones_like(input_asarray) * input.max_vals.data),
                Tensor(np.ones_like(target_asarray) * target.max_vals.data).long(),
            )
        )

        return PhiTensor(
            child=data,
            data_subjects=DSL(
                one_hot_lookup=input.data_subjects.one_hot_lookup,
                data_subjects_indexed=np.zeros_like(data),
            ),
            min_vals=minv,
            max_vals=maxv,
        )


In [7]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2)
        self.conv2 = Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2)
        self.conv3 = Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2)
        self.conv4 = Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2)
        self.conv5 = Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2)
        self.bn1 = BatchNorm2d(32)
        self.bn2 = BatchNorm2d(64)
        self.bn3 = BatchNorm2d(128)
        self.bn4 = BatchNorm2d(256)
        self.bn5 = BatchNorm2d(512)
        self.pool = MaxPool2d(kernel_size=2, stride=2)
        self.avg = AvgPool2d(3)
        self.fc = Linear(512 * 1 * 1, 2)
        
    def forward(self, x: PhiTensor):
        # First layer of CNN - running 1 at a time to debug and see if any individual componenet is failing
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = F.leaky_relu(x)
#         x = self.pool(x)
        
        # Subsequent layers
        x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
        x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
        x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
        x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
        x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
        x = self.avg(x)
        x = x.reshape((-1, 512 * 1 * 1))
        x = self.fc(x)
        return x

In [8]:
cnn_model = ConvNet()

In [12]:
from syft import PhiTensor
import numpy as np

N = 10
C_in = 3
H_in = 50
W_in = 50


input_shape = (N, C_in, H_in, W_in)
x = PhiTensor(child=np.random.randint(low=0, high=255, size=input_shape),
              data_subjects=np.zeros(input_shape),
              min_vals=0,
              max_vals=255
             )

In [15]:
def create_target_phi_tensor(input_shape):
    y = PhiTensor(child=np.random.randint(low=0, high=2, size=input_shape),
                  data_subjects=np.zeros(input_shape),
                  min_vals=0,
                  max_vals=1
             )
    return y

In [18]:
loss_fn = CrossEntropyLoss()
prediction = cnn_model(x)


target = create_target_phi_tensor(10)
loss_fn(prediction, target)

PhiTensor(child=FixedPrecisionTensor(child=47774), min_vals=<lazyrepeatarray data: 0.7607353329658508 -> shape: ()>, max_vals=<lazyrepeatarray data: 0.3790375888347626 -> shape: ()>)

Alright so the results are terrible, but since it's random information I guess that's fine...

In [19]:
prediction

PhiTensor(child=FixedPrecisionTensor(child=[[-11553   6784]
 [ -1671   8769]
 [ -2155  14245]
 [ -7785   6681]
 [ -5030  14044]
 [ -4728  12926]
 [ -1904   8442]
 [  2326  15468]
 [ -3033  17263]
 [ -7450  12054]]), min_vals=<lazyrepeatarray data: [[-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]] -> shape: (10, 2)>, max_vals=<lazyrepeatarray data: [[-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]] -> shape: (10, 2)>)

In [20]:
prediction.min_vals

<lazyrepeatarray data: [[-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]
 [-0.01747882  0.11341693]] -> shape: (10, 2)>

In [21]:
prediction.max_vals

<lazyrepeatarray data: [[-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]
 [-0.08483684  0.68978536]] -> shape: (10, 2)>

In [22]:
prediction.child.decode()

array([[-0.17628479,  0.10351562],
       [-0.02549744,  0.13380432],
       [-0.03288269,  0.21736145],
       [-0.11878967,  0.10194397],
       [-0.07675171,  0.21429443],
       [-0.07214355,  0.19723511],
       [-0.02905273,  0.1288147 ],
       [ 0.03549194,  0.23602295],
       [-0.04627991,  0.26341248],
       [-0.11367798,  0.18392944]])

In [23]:
epochs = 1
classes = 2
batch_size = 128
alpha = 0.002
device = 'cpu'

model = ConvNet().to(device)

In [24]:
criterion = torch.nn.CrossEntropyLoss()

In [25]:
optimizer = torch.optim.Adamax(model.parameters(), lr=alpha)

In [26]:
def create_phi_tensor():
    return PhiTensor(
        child=np.random.randint(0, 255, (50, 50, 3)),
        data_subjects=np.ones((50, 50, 3)) * np.random.choice([0, 1]),
        min_vals=0,
        max_vals=255
    )

In [33]:
loader_train = [(create_phi_tensor(), create_target_phi_tensor(1)) for i in range(10)]

In [34]:
from tqdm import tqdm

In [35]:
from syft.core.adp.data_subject_ledger import DataSubjectLedger
from syft.core.adp.ledger_store import DictLedgerStore
from typing import Any


ledger_store = DictLedgerStore()
user_key = b"1231"
ledger = DataSubjectLedger.get_or_create(store=ledger_store, user_key=user_key)

def get_budget_for_user(*args: Any, **kwargs: Any) -> float:
    return 999999

def deduct_epsilon_for_user(*args: Any, **kwargs: Any) -> bool:
    return True

Creating new Ledger


In [36]:
pb_loss_func = CrossEntropyLoss()

In [37]:
total_step = len(loader_train)
published_output_list = []
for epoch in range(epochs):
    for i, (images, labels) in tqdm(enumerate(loader_train)):        
        # Forward pass
        outputs = model(images)
        
        loss = pb_loss_func(outputs, labels) # we shouldn't need a [0]  here
        print("Loss: ",loss.decode())
        
        
#         published_output = outputs.publish(
#             get_budget_for_user=get_budget_for_user, 
#             deduct_epsilon_for_user=deduct_epsilon_for_user, 
#             ledger=ledger, 
#             sigma=1000
#         ).decode()
        
        
        
        # REMOVE THIS PART- I put it here just to see if the rest would work
        # published_output /= published_output.max()
        #loss = criterion(torch.Tensor(published_output), labels) # we shouldn't need a [0]  here
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))

0it [00:00, ?it/s]

Loss:  0.5390472412109375


AttributeError: 'PhiTensor' object has no attribute 'backward'

## THIS IS THE SOURCE OF THE ERROR:
https://stackoverflow.com/questions/48377214/runtimeerror-dimension-out-of-range-expected-to-be-in-range-of-1-0-but-go